In [3]:
import spacy
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd

#Loading Spacy English Library
nlp = spacy.load('en')
#DEFINING NLP ENGLISH LIB
def parser(input):
    return nlp(input)


#DEFINING VARIABLES AND VALUES
input_text = "input-text.txt"

#TAKING INPUT TEXT FROM FILE
f = open(input_text,'r')
raw_text = f.read()
# print(raw_text)

#SENTENCE AND WORD TOKENIZATION
input_text_sent_tokenize = sent_tokenize(raw_text)
# input_text_word_tokenize = word_tokenize(input_text_sent_tokenize[1])
# print(input_text_sent_tokenize)
# print(input_text_word_tokenize)

input_sentence = input_text_sent_tokenize[1]
parsedText1 = parser(input_sentence)

#QUESTION NER
def question_tag(enty):
    if enty in ("ORGANIZATION","LOCATION"):
        words.insert(0,"Where")
    elif enty in ("TIME","DATE"):
        words.insert(0,"When")
#     elif enty == "ownership":
#         words.insert(0,"Whose")
    elif enty == "PERSON":
        words.insert(0,"Who")
#     elif enty == "object":
#         words.insert(0,"Whom")
#     elif enty == "choice":
#         words.insert(0,"Which")
#     elif enty == "reason":
#         words.insert(0,"Why")
    else:
        words.insert(0,"What")
        
        #CREATE DATAFRAME FOR DEPENDENCY PARSER
data = []
for token in parsedText1:
#     print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])
    data.append([token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights]])
    
df = pd.DataFrame(data,columns=['word', 'dep', 'head_word','left','right'])
# print(df)
# df.style.applymap(apply_highlight)

#EXTRACTING SENTENCES FOR QG
words = []
a = df.dep
for i in range(0,len(a)):
    if a[i] == "ROOT":
        for j in df.iloc[i,3]:
            words.append(j)
        words.append(df.iloc[i,0])
        for j in range (i+1,len(a)):
            if df.iloc[i,0] == df.iloc[j,2]:
                words.append(df.iloc[j,0])
            else:
                break
        if a[i+1] == "aux":
            for k in df.iloc[i,4]:
                if k not in (".",","):
                    for ind,j in enumerate(df['word']):
                        if k == j:
                            if df.iloc[ind,2] == df.iloc[i,0]:
                                for z in range(i+1,ind):
                                    words.append(df.iloc[z,0])
                                words.append(df.iloc[ind,0])
for i in range(0,len(words)):
    if words[i] == "is":
        words.pop(i)
        words.insert(0,"is")
        break
    elif words[i] == "are":
        words.pop(i)
        words.insert(0,"are")
        break

        
        #COMPOUNDING SENTENCES
j = 0
compound_list = []
for i in df.dep:
    
    if i in ('compound','amod'):
        a = df.iloc[j,:]
        compound_list.append(a['word'] +' ' +a['head_word'])
    j = j+1
# print(compound_list)

#COMPOUNDING WORDS REPLACEMENT
for idx1, val1 in enumerate(words):
    for idx2, val2 in enumerate(compound_list):
        if val1 in val2:
            words[idx1] = compound_list[idx2]
#             print(words)
            break
            

import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

stanford_classifier = '/Users/ronak/Downloads/english.muc.7class.distsim.crf.ser.gz'
stanford_ner_path = '/Users/ronak/Downloads/stanford-ner-2018-10-16/stanford-ner.jar'
st = StanfordNERTagger(stanford_classifier, stanford_ner_path)


def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk


token = word_tokenize(input_sentence)

classified = st.tag(token)

ne_tagged_sent = classified
named_entities = get_continuous_chunks(ne_tagged_sent)


ner = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]
print(ner)
# print(words)

#FETCH REQUIRED NER
s = []
for indx1, i in enumerate(ner):
    for indx2, j in enumerate(words):
        if j in i:
            ner.pop(indx1)

for i in range(0,len(ner)):
    s = ner[i][1]

#ADD QUESTION
question_tag(s)
        
        

words.append("?")
print(words)

question = ' '.join(words)
question

[]
['What', 'lungs', 'begin', 'to', 'develop', '?']


'What lungs begin to develop ?'